In [35]:
#Importamos las librerias necesarias para el análisis de datos 
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os
import sys
#Importamos plotly para la visualización de los datos
import plotly.express as px
import plotly.graph_objects as go

In [36]:
df = pd.read_csv(r'C:\Users\spide\Desktop\Bootcamp\mi_entorno\Modulo_3\proyecto_final_modulo_3\vehiculos-de-segunda-mano-sample.csv',encoding = 'UTF-8')

In [37]:
df.shape

(100000, 28)

In [38]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 28 columns):
 #   Column                  Non-Null Count   Dtype  
---  ------                  --------------   -----  
 0   vehicle_type            100000 non-null  object 
 1   make                    100000 non-null  object 
 2   model                   100000 non-null  object 
 3   version                 99842 non-null   object 
 4   fuel                    96810 non-null   object 
 5   year                    100000 non-null  int64  
 6   kms                     99189 non-null   float64
 7   power                   99709 non-null   float64
 8   doors                   0 non-null       float64
 9   shift                   98982 non-null   object 
 10  color                   0 non-null       float64
 11  photos                  100000 non-null  int64  
 12  description             55810 non-null   object 
 13  price                   100000 non-null  int64  
 14  currency             

## Selección de variables que usaremos para el estudio

In [39]:
df.columns

Index(['vehicle_type', 'make', 'model', 'version', 'fuel', 'year', 'kms',
       'power', 'doors', 'shift', 'color', 'photos', 'description', 'price',
       'currency', 'location', 'publish_date', 'update_date', 'dealer_name',
       'dealer_description', 'dealer_address', 'dealer_zip_code',
       'dealer_city', 'dealer_country_code', 'dealer_is_professional',
       'dealer_website', 'dealer_registered_at', 'date'],
      dtype='object')

* Eliminaremos *vehicle_type* debido a que solo tiene el valor único car.
* Eliminaremos *doors* debido a que tiene un 100% de valores nulos, al igual que *colors*.
* Eliminaremos *currency* debido a que solo es el tipo de moneda.
* *update_date* la borraremos debido a que no nos da información para el análisis.
* Eliminaremos *dealer_name*, *dealer_description*, *dealer_city*, *dealer_zip_code*, *dealer_city*, *dealer_country_code*, *dealer_is_professional*, *dealer_website*, *dealer_registered_at*, *date* porque no nos aportan información en nuestro análisis.

In [40]:
df = df.drop(['vehicle_type','doors','color','photos', 'description','currency','update_date','dealer_name','dealer_description', 'dealer_is_professional','dealer_website','dealer_registered_at', 'date'], axis=1)

## Valores duplicados


In [41]:
df.duplicated().sum() #filas duplicadas

57295

In [42]:
df = df.drop_duplicates()

## Valores nulos

In [43]:
df.isnull().sum()

make                      0
model                     0
version                  73
fuel                   1280
year                      0
kms                     250
power                    83
shift                   697
price                     0
location                  0
publish_date              0
dealer_address           27
dealer_zip_code          27
dealer_city              27
dealer_country_code       0
dtype: int64

In [44]:
((df.isnull().sum())/df.shape[0])*100

make                   0.000000
model                  0.000000
version                0.170940
fuel                   2.997307
year                   0.000000
kms                    0.585412
power                  0.194357
shift                  1.632127
price                  0.000000
location               0.000000
publish_date           0.000000
dealer_address         0.063224
dealer_zip_code        0.063224
dealer_city            0.063224
dealer_country_code    0.000000
dtype: float64

### Valores nulos de tipo de combustible

In [45]:
# Ajustar las opciones de visualización de pandas
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', None)

# Mostrar el resultado completo
df[df['fuel'].isnull()][['make','model']].value_counts()

make           model             
Toyota         C-HR                  253
               Corolla                62
Ford           Puma                   53
Fiat           500                    44
Mazda          2                      39
KIA            Ceed                   39
Lexus          NX                     39
Toyota         Auris                  37
               Yaris                  36
KIA            Sportage               35
Hyundai        IONIQ                  29
Renault        Arkana                 29
Lexus          ES                     27
SEAT           Leon                   25
Lexus          UX                     25
KIA            Niro                   22
Citroen        C3                     22
Lexus          RX                     21
               IS                     19
BMW            Serie 4                19
Hyundai        Tucson                 17
CUPRA          Leon                   17
Lexus          RC                     16
Fiat           Panda   

In [46]:
df = df.dropna(subset=['kms', 'version'])

In [47]:
((df.isnull().sum())/df.shape[0])*100
    

make                   0.000000
model                  0.000000
version                0.000000
fuel                   2.968241
year                   0.000000
kms                    0.000000
power                  0.179321
shift                  1.625690
price                  0.000000
location               0.000000
publish_date           0.000000
dealer_address         0.063706
dealer_zip_code        0.063706
dealer_city            0.063706
dealer_country_code    0.000000
dtype: float64

In [48]:
# Lista de variables a rellenar
variables = ['fuel', 'power', 'shift']

# Función para calcular la moda o mediana por modelo
def calculate_stat_by_model(df, var):
    if var == 'power':
        return df.groupby('model')[var].transform(lambda x: x.median() if not pd.isna(x.median()) else None)
    else:
        return df.groupby('model')[var].transform(lambda x: x.mode()[0] if not x.mode().empty else None)

# Función para rellenar los valores nulos
def fill_missing_values(df, var):
    stat_by_model = calculate_stat_by_model(df, var)
    df[var] = df[var].fillna(stat_by_model)
    return df

# Aplicar la función para cada variable
for var in variables:
    df = fill_missing_values(df, var)


C:\Users\spide\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\numpy\lib\nanfunctions.py:1215: RuntimeWarning: Mean of empty slice
  return np.nanmean(a, axis, out=out, keepdims=keepdims)


In [49]:
df.isnull().sum()    

make                    0
model                   0
version                 0
fuel                    3
year                    0
kms                     0
power                   1
shift                   1
price                   0
location                0
publish_date            0
dealer_address         27
dealer_zip_code        27
dealer_city            27
dealer_country_code     0
dtype: int64

In [50]:
#Estos los arreglaremos poniendole gasolina porque lo hemos busucado en internet
df['fuel'] = df['fuel'].fillna('Gasolina')

In [51]:
df[df['power'].isnull()][['make','model']].value_counts() #Vemos los valores nulos de la variable power que nos quedan

make     model  
Citroen  ë-Jumpy    1
Name: count, dtype: int64

In [52]:
#Buscando en internet tiene 100 kW
df['power'] = df['power'].fillna(100.0)

In [53]:
df[df['shift'].isnull()][['make','model','version']].value_counts() #Vemos los valores nulos de la variable shift que nos quedan

make   model   version             
Lotus  Esprit  Coche de competición    1
Name: count, dtype: int64

In [54]:
#Buscando en internet el vehículo es manual
df['shift']=df['shift'].fillna('manual')

In [55]:
df.isnull().sum()   

make                    0
model                   0
version                 0
fuel                    0
year                    0
kms                     0
power                   0
shift                   0
price                   0
location                0
publish_date            0
dealer_address         27
dealer_zip_code        27
dealer_city            27
dealer_country_code     0
dtype: int64

In [56]:
df[df['dealer_zip_code'].isnull()]['location'].value_counts()

location
Elx/Elche                                     21
Parque Empresarial San Fernando de Henares     5
Collado Villalba                               1
Name: count, dtype: int64

In [57]:
#Pasamos a texto el valor de dealer_zip_code por aquellos que empiezen por 0
df['dealer_zip_code'] = df['dealer_zip_code'].astype(str)

In [58]:
df.loc[df['location'] == 'Parque Empresarial San Fernando de Henares', 'dealer_zip_code'] = df.loc[df['location'] == 'Parque Empresarial San Fernando de Henares', 'dealer_zip_code'].fillna('28830')
df.loc[df['location'] == 'Collado Villalba', 'dealer_zip_code'] = df.loc[df['location'] == 'Collado Villalba', 'dealer_zip_code'].fillna('28400')
df.loc[df['location'] == 'Elx/Elche', 'dealer_zip_code'] = df.loc[df['location'] == 'Elx/Elche', 'dealer_zip_code'].fillna('03203')

In [59]:
df.isnull().sum()   

make                    0
model                   0
version                 0
fuel                    0
year                    0
kms                     0
power                   0
shift                   0
price                   0
location                0
publish_date            0
dealer_address         27
dealer_zip_code         0
dealer_city            27
dealer_country_code     0
dtype: int64

In [60]:
df = df.dropna(subset=['dealer_address'])

In [61]:
df.isnull().sum()

make                   0
model                  0
version                0
fuel                   0
year                   0
kms                    0
power                  0
shift                  0
price                  0
location               0
publish_date           0
dealer_address         0
dealer_zip_code        0
dealer_city            0
dealer_country_code    0
dtype: int64

In [30]:
#Guardar df en csv 
#df.to_csv('car_price.csv', index=False)

## Latitud, Longitud y Provincia

In [67]:
import pandas as pd
from geopy.geocoders import Nominatim
from geopy.exc import GeocoderTimedOut, GeocoderServiceError
import time

def clean_address(address):
    """
    Limpia y formatea la dirección
    """
    if pd.isna(address):
        return ""
    # Convertir a string si no lo es
    address = str(address)
    # Eliminar caracteres especiales y espacios extras
    address = address.replace("NaN", "").strip()
    return address

def get_coordinates(address, city, country_code):
    """
    Get latitude and longitude from address using Nominatim geocoder
    """
    # Limpiar los componentes de la dirección
    address = clean_address(address)
    city = clean_address(city)
    country_code = clean_address(country_code)
    
    # Si no hay suficiente información, retornar None
    if not (address and city and country_code):
        print(f"Información incompleta: {address}, {city}, {country_code}")
        return None, None
    
    # Crear la dirección completa
    full_address = f"{address}, {city}, {country_code}"
    print(f"Buscando dirección: {full_address}")  # Para depuración
    
    # Initialize the geocoder
    geolocator = Nominatim(user_agent="my_vehicles_app")
    
    try:
        time.sleep(1)  # Respetar límites de la API
        
        # Intentar primero con la dirección completa
        location = geolocator.geocode(full_address)
        
        # Si falla, intentar solo con ciudad y país
        if not location:
            print("Intentando con ciudad y país...")
            location = geolocator.geocode(f"{city}, {country_code}")
        
        if location:
            print(f"Encontrado: {location.latitude}, {location.longitude}")
            return location.latitude, location.longitude
        else:
            print("No se encontró la ubicación")
            return None, None
            
    except (GeocoderTimedOut, GeocoderServiceError) as e:
        print(f"Error de geocodificación: {str(e)}")
        return None, None

def get_province_from_coordinates(lat, long):
    """
    Obtiene la provincia utilizando las coordenadas de latitud y longitud
    """
    if lat is None or long is None:
        return 'Unknown'
    
    geolocator = Nominatim(user_agent="my_vehicles_app")
    try:
        time.sleep(1)  # Respetar límites de la API
        location = geolocator.reverse((lat, long), timeout=10).raw['address']
        print(f"Provincia encontrada: {location.get('state', 'Unknown')}")  # 'state' puede variar dependiendo del país
        return location.get('state', 'Unknown'),  # 'state' puede variar dependiendo del país
    except (GeocoderTimedOut, GeocoderServiceError, KeyError) as e:
        print(f"Error al obtener la provincia: {str(e)}")
        return 'Unknown'

def add_coordinates_and_state_to_df(df):
    """
    Add latitude, longitude, and state columns to the DataFrame
    """
    # Initialize new columns
    df['lat'] = None
    df['long'] = None
    df['state'] = None
    
    # Iterate through rows
    for idx, row in df.iterrows():
        print(f"\nProcesando fila {idx + 1}/{len(df)}")
        
        # Imprimir los valores para depuración
        print(f"Dirección: {row.get('dealer_address', 'No disponible')}")
        print(f"Ciudad: {row.get('dealer_city', 'No disponible')}")
        print(f"País: {row.get('dealer_country_code', 'No disponible')}")
        
        # Obtener latitud y longitud
        lat, long = get_coordinates(
            row.get('dealer_address', ''),
            row.get('dealer_city', ''),
            row.get('dealer_country_code', '')
        )
        
        # Obtener la provincia si se encontraron coordenadas
        state = get_province_from_coordinates(lat, long)
        
        # Actualizar el DataFrame
        df.at[idx, 'lat'] = lat
        df.at[idx, 'long'] = long
        df.at[idx, 'state'] = state
    
    return df


In [63]:
df_coordenadas = df[['dealer_address', 'dealer_city', 'dealer_country_code']]

In [64]:
# Dividir el dataframe en 2 partes
df2, df3 = np.array_split(df_coordenadas, 2)

# Mostrar las formas de los dataframes resultantes
print(df2.shape)
print(df3.shape)


(21178, 3)
(21177, 3)


C:\Users\spide\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\numpy\core\fromnumeric.py:59: FutureWarning: 'DataFrame.swapaxes' is deprecated and will be removed in a future version. Please use 'DataFrame.transpose' instead.
  return bound(*args, **kwds)


In [65]:
df2.reset_index(drop=True, inplace=True)

In [70]:
df_jaume.head(10)

NameError: name 'df_jaume' is not defined